# First things first
* Click **File -> Save a copy in Drive** and click **Open in new tab** in the pop-up window to save your progress in Google Drive.
* Click **Runtime -> Change runtime type** and select **GPU** in Hardware accelerator box to enable faster GPU training.


# AWS


In [1]:
try:
    import google.colab
    from google.colab.output import clear as clear_output
    IN_COLAB = True
except:
    IN_COLAB = False
    from IPython.display import clear_output as clear_output



# if IN_COLAB:
#     print("Downloading Colab files")
#     ! shred -u setup_google_colab.py
#     ! wget https://raw.githubusercontent.com/hse-aml/bayesian-methods-for-ml/master/setup_google_colab.py -O setup_google_colab.py
#     import setup_google_colab
#     setup_google_colab.load_data_week5()

In [2]:
! pip install awscli
! pip install pytorch-crf

In [3]:
# Please refer here for your aws key:
# https://paper.dropbox.com/doc/dvc-Tutorial--A3qgCNFVX_OyflleiwN8BlKrAg-uvhb37yY0U8nnb7sjAfey


# region = us-west-2
# output = json

In [ ]:
! aws configure
clear_output()

In [8]:
! aws s3 cp  "s3://cinnamontwbootcamp/bootcamp2020/Preprocessing/" "./" --recursive
# ! aws s3 cp  "s3://research-tw/bootcamp2020/Preprocessing/" "./" --recursive 
# clear_output()

download: s3://cinnamontwbootcamp/bootcamp2020/Preprocessing/dictionary/charToIndex.json to dictionary/charToIndex.json
download: s3://cinnamontwbootcamp/bootcamp2020/Preprocessing/data/test.txt to data/test.txt
download: s3://cinnamontwbootcamp/bootcamp2020/Preprocessing/data/gold/cityu_test_gold.utf8.txt to data/gold/cityu_test_gold.utf8.txt
download: s3://cinnamontwbootcamp/bootcamp2020/Preprocessing/data/gold/as_testing_gold.txt to data/gold/as_testing_gold.txt
download: s3://cinnamontwbootcamp/bootcamp2020/Preprocessing/data/gold/as_training_words.txt to data/gold/as_training_words.txt
download: s3://cinnamontwbootcamp/bootcamp2020/Preprocessing/data/gold/cityu_training_words.txt to data/gold/cityu_training_words.txt
download: s3://cinnamontwbootcamp/bootcamp2020/Preprocessing/dictionary/indexToChar.json to dictionary/indexToChar.json
download: s3://cinnamontwbootcamp/bootcamp2020/Preprocessing/dictionary/indexToLabel.json to dictionary/indexToLabel.json
download: s3://cinnamontwb

In [9]:
! ls  data
! ls dictionary
! ls

gold  test  test.txt  train
charToIndex.json  indexToChar.json  indexToLabel.json  labelToIndex.json
data  dictionary  models  sample_data


# Preprocessing data



In [17]:
file_name = './data/train/as_training.txt'
data_train_text = open(file_name, encoding='utf-8-sig').readlines()


In [18]:
#BNE, S
from tqdm import tqdm

data_train_tag = []


for line in tqdm(data_train_text):
    
    tmp = ''
    for word in line.split():
        
        # Single word
        if len(word)==1:  tmp += 'S'
            
        else:
            for i in range(len(word)):
                
                # Begin
                if i == 0:  tmp += 'B'
                    
                # End
                elif i == len(word)-1:  tmp += 'E'
                    
                else:  tmp += 'N'
                    
    data_train_tag.append(tmp)

100%|██████████| 708953/708953 [00:04<00:00, 160288.75it/s]


In [19]:
# Visualize labels

idx = 777


line = data_train_text[idx]
tag = data_train_tag[idx]

print("\t".join(line.split()),
      "E\t".join("S\t".join(tag.split("S")).split("E")),
      sep="\n")




其實	如果	我們	能	回答	這	個	大	難題	，
BE	BE	BE	S	BE	S	S	S	BE	S	


In [20]:
train = [line.replace('\t','') for line in data_train_text]

max_idx, max_line = max(enumerate(train),
                        key=lambda x: len(x[-1]))

max_idx


698437

In [21]:
with open('./data/train.txt','w',encoding='utf8') as f:
    for line in train[:100000]:
        f.write(line.strip()+'\n')

In [22]:
with open('./data/labels.txt','w',encoding='utf8') as f:
    for line in data_train_tag[:100000]:
        f.write(line.strip()+'\n')

# Model Training

In [23]:
import random
import json

import warnings
from collections import Counter
from torchcrf import CRF
import numpy as np
from sklearn.metrics import classification_report
warnings.filterwarnings("ignore")



In [24]:
class ModelConfig(object):
    embeddingSize = 300 
    
    hiddenSizes =  128 
    
class Config(object):
    sequenceLength = 188  
    batchSize = 32
    
    train_source = "./data/train.txt"
    label_source = "./data/labels.txt"
    
    rate = 0.8  
    
    model = ModelConfig()

config = Config()

In [25]:
class Dataset(object):
    def __init__(self, config):
        self._trainSource = config.train_source
        self._labelSource = config.label_source
        
        self._sequenceLength = config.sequenceLength
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self.trainTrains = []
        self.trainLabels = []
        self.trainMasks = []
        
        self.evalTrains = []
        self.evalLabels = []
        self.evalMasks = []
        
        self._charToIndex = {}
        self._indexToChar = {}
        self._labelToIndex = {}
        self._indexToLabel = {}
        
        self.vocab_size = None
        self.target_size = None
        
    def _readData(self, trainPath, labelPath):
        trains = open(trainPath,encoding='utf8').readlines()
        labels = open(labelPath,encoding='utf8').readlines()
        
        trains_c = [s.replace('\n','') for s in trains]
        labels_c = [t.replace('\n','') for t in labels]
        return trains_c, labels_c

    def _trainProcess(self, train, sequenceLength, charToIndex):
        trainVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        

        if len(train) < sequenceLength:
            sequenceLen = len(train)
            datalen=len(train)
        else:
            datalen=sequenceLength
        
        for i in range(sequenceLen):
            if train[i] in charToIndex:
                trainVec[i] = charToIndex[train[i]]
            else:
                trainVec[i] = charToIndex["UNK"]

        return trainVec, datalen
    
    def _labelProcess(self, label, sequenceLength, labelToIndex):
        labelVec = np.zeros((sequenceLength))
        mask = np.zeros((sequenceLength))
        
        for i in range(sequenceLength):
            if i+1 <= len(label):
                labelVec[i] = labelToIndex[label[i]]
                mask[i] = 1


        return labelVec, mask

    def _genTrainEvalData(self, x, y, rate):
        trains = []
        labels = []
        lengths=[]
        masks = []
        
        for i in range(len(x)):
            trainVec, length = self._trainProcess(x[i], self._sequenceLength, self._charToIndex)
            trains.append(trainVec)
            
            labelVec, mask = self._labelProcess(y[i], self._sequenceLength, self._labelToIndex)
            labels.append(labelVec)
            
            lengths.append(length)
            masks.append(mask)
            
        trainIndex = int(len(x) * rate)
        
        trainTrains = np.asarray(trains[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        trainlengths = np.asarray(lengths[:trainIndex], dtype="int64")
        trainMasks = np.asarray(masks[:trainIndex], dtype="int64")
        
        evalTrains = np.asarray(trains[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")
        evallengths = np.asarray(lengths[trainIndex:], dtype="int64")
        evalMasks = np.asarray(masks[trainIndex:], dtype="int64")

        return trainTrains, trainLabels, evalTrains, evalLabels, trainlengths, evallengths, trainMasks, evalMasks
        
    def _genVocabulary(self, trains):
        allChars = [char for train in trains for char in train]
        
        charCount = Counter(allChars)
        sortCharCount = sorted(charCount.items(), key=lambda x: x[1], reverse=True)
        chars = [item[0] for item in sortCharCount]
        
        vocab = self._getCharV(chars)
        labels = ['pad','B', 'N', 'E', 'S']
        self._charToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToChar = dict(zip(list(range(len(vocab))), vocab))
        
        self._labelToIndex = dict(zip(labels, list(range(len(labels)))))
        self._indexToLabel = dict(zip(list(range(len(labels))), labels))
        
        self.target_size = len(labels)
        
        with open("./dictionary/charToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self._charToIndex, f)
        
        with open("./dictionary/indexToChar.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToChar, f)
            
        with open("./dictionary/labelToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self._labelToIndex, f)
        
        with open("./dictionary/indexToLabel.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToLabel, f)
            
    def _getCharV(self, chars):
        
        vocab = []
        vocab.append("pad")
        vocab.append("UNK")

        for char in chars:
            vocab.append(char)
        
        self.vocab_size = len(vocab)
        
        return vocab
            
    def dataGen(self):

        trains, labels = self._readData(self._trainSource, self._labelSource)
        self._genVocabulary(trains)
        
        trainTrains, trainLabels, evalTrains, evalLabels, trainlengths, evallengths, trainMasks, evalMasks = self._genTrainEvalData(trains, labels, self._rate)
        self.trainTrains = trainTrains
        self.trainLabels = trainLabels
        self.trainLengths = trainlengths
        self.trainMasks = trainMasks
        
        self.evalTrains = evalTrains
        self.evalLabels = evalLabels
        self.evalLengths = evallengths
        self.evalMasks = evalMasks
        
data = Dataset(config)
data.dataGen()

In [26]:
def nextBatch(x, y, l, m, batchSize):
    
    perm = np.arange(len(x))
    np.random.shuffle(perm)
    x = x[perm]
    y = y[perm]
    l = l[perm]
    m = m[perm]
    numBatches = len(x) // batchSize

    for i in range(numBatches):
        start = i * batchSize
        end = start + batchSize
        batchX = np.array(x[start: end], dtype="int64")
        batchY = np.array(y[start: end], dtype="float32")
        batchL = np.array(l[start: end], dtype="int64")
        batchM = np.array(m[start: end], dtype="int64")
            
        yield batchX, batchY, batchL, batchM

In [27]:
train_ = data.trainTrains
trainLabels = data.trainLabels
trainLengths = data.trainLengths
trainMasks = data.trainMasks

eval_ = data.evalTrains
evalLabels = data.evalLabels
evalLengths = data.evalLengths
evalMasks = data.evalMasks

In [28]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F


class Bi_Lstm_Model(torch.nn.Module) :
    def __init__(self,hidden_dim, bs, seqence_l, vocab_size, target_size, embedding_dim) :
        super(Bi_Lstm_Model,self).__init__()
        self.batchsize = bs
        self.vocab_size = vocab_size
        self.target_size = target_size
        self.seqence_l = seqence_l
        self.hidden_dim = hidden_dim
        self.embedding_dim=embedding_dim
        self.embeddings = nn.Embedding(vocab_size, self.embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(self.embedding_dim, hidden_dim // 2, dropout=0.7,
                            num_layers=2, bidirectional=True, batch_first=True)
        self.linearOut = nn.Linear(hidden_dim, target_size)
        
    def forward(self,inputs, inputs_lens) :
        self.hidden=self.init_hidden()
 
        x = self.embeddings(inputs).view(self.batchsize, inputs.shape[1], self.embeddings.embedding_dim)
        
        x = torch.nn.utils.rnn.pack_padded_sequence(x, inputs_lens, batch_first=True, enforce_sorted=False)
        x, self.hidden = self.lstm(x, self.hidden)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True, total_length=self.seqence_l)
        
        x=x.view(self.batchsize, -1, self.hidden_dim)

        x = self.linearOut(x)
        
        return x
    
    def init_hidden(self):
        return (torch.randn(4, self.batchsize, self.hidden_dim // 2).cuda(),
                torch.randn(4, self.batchsize, self.hidden_dim // 2).cuda())

In [22]:
model =Bi_Lstm_Model(config.model.hiddenSizes, config.batchSize, config.sequenceLength, data.vocab_size, data.target_size, config.model.embeddingSize).cuda()
crf = CRF(data.target_size, batch_first=True).cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [23]:
model

Bi_Lstm_Model(
  (embeddings): Embedding(4708, 300, padding_idx=0)
  (lstm): LSTM(300, 64, num_layers=2, batch_first=True, dropout=0.7, bidirectional=True)
  (linearOut): Linear(in_features=128, out_features=5, bias=True)
)

In [27]:
def train(train_data, epoch, config, model, loss_function, optimizer):
    print('epoch '+str(epoch+1)+' training')
    y_p=[]
    y_t=[]
    model.train()
    avg_loss = 0.0
    count=0
    # need a batch dataloader
    for batch in nextBatch(train_data[0], train_data[1], train_data[2], train_data[3], config.batchSize):
        input_data = Variable(torch.cuda.LongTensor(batch[0]))
        target_data = Variable(torch.cuda.LongTensor(batch[1])).squeeze(1)

        input_lens = Variable(torch.cuda.LongTensor(batch[2]))
        input_masks = Variable(torch.cuda.ByteTensor(batch[3]))

        y_pred = model(input_data, input_lens)
        model.zero_grad()
        loss = -crf(y_pred,target_data, mask=input_masks)
        avg_loss += loss.item()
        #a better metric here
        predictions = crf.decode(y_pred, mask=input_masks)
        
        y_p += [p for ps in predictions for p in ps]
        unpad_target = [t[:le] for t, le in zip(target_data.tolist(),input_lens)]
        y_t += [t for ts in unpad_target for t in ts]
        #a better metric here 
        count+=1
        if count%1000 == 0:
            print('epoch :%d iterations :%d loss :%g'%(epoch+1,config.batchSize*count,loss.item()))

        loss.backward()
        optimizer.step()

        # if count > 20:
            # break

    torch.save(model.state_dict(), './models/model_crf' + str(epoch+1)+'.pth') 
    
    print('the average loss after completion of %d epochs is %g'%((epoch+1),(avg_loss/count)))
    
    print("="*50)
    #a better metric here

    if min(y_p)==0:
        names = ['pad', 'B','N','E','S']
    else: 
        names = ['B','N','E','S']

    print(classification_report(np.array(y_t).flatten(), np.array(y_p).flatten(),
                                target_names=names))
    
    print("="*50)

    return y_t, y_p

    #a better metric here



In [28]:
def evaluate(test_data, model,model_dir,epochs, config):
    y_p=[]
    y_t=[]

    avg_loss_e=0
    count=0
    model.load_state_dict(torch.load(model_dir))
    model.eval()
    for batch in nextBatch(test_data[0],test_data[1], test_data[2], test_data[3], config.batchSize) :     
        input_data = Variable(torch.cuda.LongTensor(batch[0]))
        target_data = Variable(torch.cuda.LongTensor(batch[1])).squeeze(1)
        
        input_lens=Variable(torch.cuda.LongTensor(batch[2]))
        input_masks=Variable(torch.cuda.ByteTensor(batch[3]))

        y_pred= model(input_data, input_lens)
        model.zero_grad()
        loss = -crf(y_pred,target_data, mask=input_masks)
        avg_loss_e += loss.item()
        predictions = crf.decode(y_pred, mask=input_masks)
        
        y_p += [p for ps in predictions for p in ps]
        unpad_target = [t[:le] for t, le in zip(target_data.tolist(),input_lens)]
        y_t += [t for ts in unpad_target for t in ts]
        
        count += 1
        # if count > 20:
            # break

    print('epoch :%d eval loss :%g'%(epochs, avg_loss_e/count))

    if min(y_p)==0:
        names = ['pad', 'B','N','E','S']
    else: 
      names = ['B','N','E','S']

    print(classification_report(np.array(y_t).flatten(), np.array(y_p).flatten(),
                                target_names=names))
    
    print("="*50)

    return y_t, y_p


In [29]:
epochs=4
for epoch in range(epochs):
    t, p = train([train_, trainLabels, trainLengths, trainMasks], epoch, config, model, crf, optimizer)
    ti, pi = evaluate([eval_,evalLabels, evalLengths, evalMasks], model,'./models/model_crf' + str(epoch+1)+'.pth', epoch+1, config)

epoch 1 training
epoch :1 iterations :32000 loss :122.812
epoch :1 iterations :64000 loss :97.8616
the average loss after completion of 1 epochs is 143.827
              precision    recall  f1-score   support

         pad       0.00      0.00      0.00         0
           B       0.84      0.89      0.86    268010
           N       0.72      0.45      0.56     41280
           E       0.84      0.88      0.86    268010
           S       0.89      0.87      0.88    365696

    accuracy                           0.86    942996
   macro avg       0.66      0.62      0.63    942996
weighted avg       0.86      0.86      0.86    942996

epoch :1 eval loss :149.157
              precision    recall  f1-score   support

           B       0.85      0.90      0.87     73258
           N       0.78      0.52      0.63     16559
           E       0.85      0.90      0.88     73258
           S       0.91      0.87      0.89     91465

    accuracy                           0.87    254540
 

In [30]:
_, _ = evaluate([eval_,evalLabels, evalLengths, evalMasks], model,'./models/model_crf' + str(epoch+1)+'.pth', epoch+1, config)

epoch :4 eval loss :120.608
              precision    recall  f1-score   support

           B       0.89      0.92      0.90     73258
           N       0.82      0.63      0.71     16559
           E       0.89      0.92      0.91     73258
           S       0.92      0.91      0.91     91465

    accuracy                           0.90    254540
   macro avg       0.88      0.84      0.86    254540
weighted avg       0.89      0.90      0.89    254540



In [ ]:
# np.array(ti).shape, np.array(pi).shape

# idxs = filter(lambda x: x[-1]==0, enumerate(pi))
# list(idxs)


# Inference

In [7]:
import json
import warnings
import numpy as np
warnings.filterwarnings("ignore")

from torchcrf import CRF

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

In [30]:
#only predict
class ModelConfig(object):
    embeddingSize = 300
    
    hiddenSizes =  128
    
class Config(object):
    sequenceLength = 188  
    batchSize = 32
    
    test_source = "./test.txt"
    char2index_source = './dictionary/charToIndex.json'
    index2char_source = './dictionary/indexToChar.json'
    index2label_source = './dictionary/indexToLabel.json'
    
    model = ModelConfig()

    
config = Config()

In [31]:
class Dataset(object):
    def __init__(self, config):
        self._testSource = config.test_source
        
        self._sequenceLength = config.sequenceLength
        self._batchSize = config.batchSize
        
        self.testTests = []
        self.testMasks = []
        
        with open(config.char2index_source) as f:
            self._charToIndex = json.load(f)
        
        with open(config.index2char_source) as f:
            self._indexToChar = json.load(f)

        with open(config.index2label_source) as f:
            self._indexToLabel = json.load(f)
            
        self.target_size = len(self._indexToLabel)
        self.vocab_size = len(self._charToIndex)
        
    def _readData(self, testPath):
        tests = open(testPath,encoding='utf8').readlines()
        
        tests_c = [s.replace('\n','') for s in tests]
        return tests_c

    def _testProcess(self, test, sequenceLength, charToIndex):
        testVec = np.zeros((sequenceLength))
        mask = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        if len(test) < sequenceLength:
            sequenceLen = len(test)
            datalen=len(test)
        else:
            datalen=sequenceLength
        
        for i in range(sequenceLen):
            if test[i] in charToIndex:
                testVec[i] = charToIndex[test[i]]
            else:
                testVec[i] = charToIndex["UNK"]
            if i+1 <= len(test):
                mask[i] = 1        
        

        return testVec, datalen, mask

    def _genTestData(self, x):
        
        tests = []
        lengths=[]
        masks = []
        
        for i in range(len(x)):
            testVec, length, mask = self._testProcess(x[i], self._sequenceLength, self._charToIndex)
            tests.append(testVec)
            
            lengths.append(length)
            masks.append(mask)
        
        testTests = np.asarray(tests, dtype="int64")
        testlengths = np.asarray(lengths, dtype="int64")
        testMasks = np.asarray(masks, dtype="int64")

        return testTests, testlengths, testMasks
            
    def dataGen(self):
        tests= self._readData(self._testSource)
        
        testTests, testlengths, testMasks = self._genTestData(tests)
        
        self.testTests = testTests
        self.testLengths = testlengths
        self.testMasks = testMasks
        

        
data = Dataset(config)
data.dataGen()

In [32]:
def nextBatch(x, l, m, batchSize):
    numBatches = len(x) // batchSize

    for i in range(numBatches):
        start = i * batchSize
        end = start + batchSize
        batchX = np.array(x[start: end], dtype="int64")
        batchL = np.array(l[start: end], dtype="int64")
        batchM = np.array(m[start: end], dtype="int64")
            
        yield batchX, batchL, batchM

In [33]:
test_ = data.testTests
testLengths = data.testLengths
testMasks = data.testMasks

In [34]:
#model_load
predict_model = Bi_Lstm_Model(config.model.hiddenSizes, 
                              config.batchSize, 
                              config.sequenceLength, 
                              data.vocab_size, 
                              data.target_size, 
                              config.model.embeddingSize).cuda()

predict_model.load_state_dict(torch.load('./models/model_crf4.pth'))
predict_model.eval()
crf = CRF(data.target_size, batch_first=True).cuda()

results=[]

for batch in nextBatch(test_, testLengths, testMasks, config.batchSize) :     
    input_data = Variable(torch.cuda.LongTensor(batch[0]))
    input_lens = Variable(torch.cuda.LongTensor(batch[1]))
    input_masks = Variable(torch.cuda.ByteTensor(batch[2]))

    y_pred = predict_model(input_data, input_lens)
    predict_model.zero_grad()
    predictions = crf.decode(y_pred, mask=input_masks)
    results += predictions

print('Inference Done.')   

Inference Done.


In [35]:
#SPLIT!!!!!!!!!
testdata = open('./test.txt', encoding = 'utf8').readlines()
tags = []
for r in results:
    tmp = []
    for t in r:
        tmp.append(data._indexToLabel[str(t)])
    tags.append(tmp)

word_seg=[]
for tag, sentence in zip(tags,testdata):
    tmp = ''
    for t, s in zip(tag, sentence):
        if t == 'S':
            tmp = tmp + s + ' '
        elif t == 'E':
            tmp = tmp + s + ' '
        else:
            tmp = tmp + s
    word_seg.append(tmp)

In [36]:
word_seg


['「 練 得 銅皮鐵骨 」 露宿 早慣 蚊叮 ',
 '本港 約 有 450 至 60 0名 露宿 者 ， 其中 近 四分之一 ， 即 約 150人 露宿 在 深水埗 。 ',
 '有 外展社工稱 ， 露宿 者 日間 多 到 商場 等 冷氣場 所 避暑 ， 流連 至 晚上 11 、 12 時 ， 才 用 紙 皮 在 公園 外 「 打地鋪 」 ， 他們 早已 「 練 得 一 身 銅皮鐵骨 」 ， 徹夜 被 蚊叮 也 習以 為常 ， 但 社工 在 炎夏 仍 會 頻頻給 他們 派發 蚊香 。 ',
 '基督教 關懷 無家 者 協會 的 外展 社工 ， 過去 一直 有 探訪 李鄭屋 遊樂場 外 的 露宿 者 ， 該 會 總幹事 賴淑芬 說 ， 該 處 的 露宿 者 只 有數 人 ， 且 流動性 很 大 。 ',
 '不管 被 多少 蚊叮 也 沒 什感 覺 ',
 '她 指 這些 露宿 者 日間 都 會 流連於 冷氣場所 ， 晚上 才 到 遊樂場 露宿 ， 但 礙於 遊樂場 晚上 關門 ， 他們 只 可 在 外圍 「 打地鋪 」 。 ',
 '由於 附近 多 草叢 ， 晚上 人 流少 ， 部分 露宿 者 又 因為 個人 衛生 問題 ， 總成為 蚊叮 對象 。 ',
 '但 露宿 者 早已 習慣 被 蚊叮 ， 不管 被 多少 蚊叮 也 沒 什 感覺 。 ',
 '不過 ， 有 些 露宿 者 一樣 被 蚊叮 至 痕癢 入心 ， 要 不斷 換位 找 個 較 少蚊 的 地方 睡覺 ， 故 徹夜 難 眠 。 ',
 '賴淑芬 指區 內 露宿 者 之 家 經常 「 爆滿 」 ， 他們 未必 有 位 入住 ﹔ ',
 '有 些 人 又 不 願 遵守 宿舍 規矩 ， 即使 捱熱 兼 「 周身癢 」 ， 也 寧願 露宿 街頭 。 ',
 '但 居於 附近 的 街坊 批評 有關 露宿 者 不 注重 衛生 ， 促請 政府 助 他們 搬走 。 ',
 '然而 ， 在 南昌街 與 通州街 交界 天 橋底 生活 、 依靠 綜援 過活 的 露宿 者 張 先生 稱 ， 露宿 者 也 很 自律 ， 將 雜物 棄置 發泡 膠箱 內 ， 每 晚 輪流 將 垃圾箱 搬到 附近 垃圾站 ， 免 滋生 蚊蟲 。 ',
 '「 在 街頭 露宿 多 年 ， 什麼 都 習慣 了 ， 但 即使 被 蚊叮 ， 也 沒辦法 。 